In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
# for ensemble method use
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
#TO DO-
#One-R
#under or over sampling


# height removed
Test_Target = ["SUSPECT_ARRESTED_FLAG", "STOP_WAS_INITIATED",
                 "OBSERVED_DURATION_MINUTES", "SUSPECTED_CRIME_DESCRIPTION",
                 "STOP_DURATION_MINUTES", "FIREARM_FLAG",
                 "SUSPECT_RACE_DESCRIPTION",
                 "DEMEANOR_OF_PERSON_STOPPED", "SUSPECT_BODY_BUILD_TYPE","FRISKED_FLAG","SEARCHED_FLAG","STOP_LOCATION_BORO_NAME"]
Test1 = pd.read_excel("sqf2022.xlsx", usecols=Test_Target)

Feature_test=["FIREARM_FLAG","FRISKED_FLAG","SEARCHED_FLAG"]

Test1['FIREARM_FLAG'] = Test1['FIREARM_FLAG'].replace('(null)', 'No')
Test1.replace('(null)', np.nan, inplace=True)

lb = LabelBinarizer()
Test1["SUSPECT_ARRESTED_FLAG"] = lb.fit_transform(Test1["SUSPECT_ARRESTED_FLAG"])
Test1["FIREARM_FLAG"] = lb.fit_transform(Test1["FIREARM_FLAG"])
Test1["FRISKED_FLAG"] = lb.fit_transform(Test1["FRISKED_FLAG"])
Test1["SEARCHED_FLAG"] = lb.fit_transform(Test1["SEARCHED_FLAG"])


Test1["SUSPECT_ARRESTED_FLAG"].fillna(Test1["SUSPECT_ARRESTED_FLAG"].median,inplace=True)
Test1["FIREARM_FLAG"].fillna(Test1["FIREARM_FLAG"].median,inplace=True)
Test1["FRISKED_FLAG"].fillna(Test1["FRISKED_FLAG"].median,inplace=True)
Test1["SEARCHED_FLAG"].fillna(Test1["SEARCHED_FLAG"].median,inplace=True)



ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [32]:
#One Hot-Encoding
OneHot_Encode_feature = ["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION"]
Label_En = OneHotEncoder(handle_unknown="ignore")
Feature_Encoded = Label_En.fit_transform(Test1[OneHot_Encode_feature])
Label_df = pd.DataFrame(Feature_Encoded.toarray(), columns=Label_En.get_feature_names_out(OneHot_Encode_feature))


In [33]:
#label encoding
LE=LabelEncoder()
Test1['SUSPECT_BODY_BUILD_TYPE']=LE.fit_transform(Test1["SUSPECT_BODY_BUILD_TYPE"])
Test1['SUSPECT_BODY_BUILD_TYPE'].fillna(Test1['SUSPECT_BODY_BUILD_TYPE'].mean, inplace=True)

In [34]:
cat_feature=list(Label_df.columns)+Feature_test
Test1 = pd.concat([Test1, Label_df], axis=1)
# drop the original categorical feature column
Test1 = Test1.drop(columns=OneHot_Encode_feature)

train_data,test_data=train_test_split(Test1,test_size=0.2,random_state=42)

#classi
X_train = train_data[cat_feature] # training features_for Random forest
y_train = train_data['SUSPECT_ARRESTED_FLAG'] # training target variable for random forest

X_test = test_data[cat_feature] # testing features
y_test = test_data['SUSPECT_ARRESTED_FLAG'] # testing target variable

In [35]:
Model_N=MultinomialNB()
Model_N.fit(X_train,y_train)
N_prediction=Model_N.predict(X_test)
train_N_prediction=Model_N.predict(X_train)
N_train_accuracy=accuracy_score(y_train,train_N_prediction)
N_accuracy=accuracy_score(y_test,N_prediction)

print('Train_Accuracy:', N_accuracy)
print('Accuracy:', N_train_accuracy)


Train_Accuracy: 0.8391261171797418
Accuracy: 0.8370168032447645


In [36]:
model_R = RandomForestClassifier(random_state=42)
#model_R.fit(train_data[Feature_test], train_data["SUSPECT_ARRESTED_FLAG"])

#with only caterogical data but transform to numeric/ with both

model_R.fit(X_train,y_train)
Prediction=model_R.predict(X_test)

train_Prediction =model_R.predict(X_train)


train_accuracy=accuracy_score(y_train,train_Prediction)

accuracy = accuracy_score(y_test, Prediction)


print('Train_Accuracy:', train_accuracy)
print('Accuracy:', accuracy)


Train_Accuracy: 0.8673950831884778
Accuracy: 0.8679245283018868


In [37]:
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions on the testing and training data, and calculate RMSE
y_test_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)

print('Testing RMSE:', test_rmse)
print('Training RMSE:', train_rmse)

Testing RMSE: 0.34058471641017063
Training RMSE: 0.345491521615495


In [38]:
logreg = LogisticRegression(max_iter=1000,random_state=42)
logreg.fit(X_train, y_train)
log_predict=logreg.predict(X_test)
log_score=accuracy_score(y_test,log_predict)

train_log_predict=logreg.predict(X_train)
train_log_score=accuracy_score(y_train,train_log_predict)

print("Accuracy:",log_score)
print("Train Accuract:",train_log_score)

Accuracy: 0.8659384309831182
Train Accuract: 0.8611042132273818


In [39]:
Dec_model=DecisionTreeClassifier(random_state=42)
Dec_model.fit(X_train,y_train)
Dec_predict=Dec_model.predict(X_test)
Train_Dec_predict=Dec_model.predict(X_train)
Dec_accuracy=accuracy_score(y_test,Dec_predict)
Train_Dec_accuracy=accuracy_score(y_train,Train_Dec_predict)
print("Decision Tree score: ",Dec_accuracy)
print("Train_Decision Tree score: ",Train_Dec_accuracy)


Decision Tree score:  0.8652763985435287
Train_Decision Tree score:  0.8673950831884778


In [40]:
dummy_class = DummyClassifier(strategy="most_frequent")
dummy_class.fit(X_train,y_train)
y_predict=dummy_class.predict(X_test)
y_train_predict=dummy_class.predict(X_train)
Dummy_accuracy = accuracy_score(y_test, y_predict)
Dummy_train_accuracy = accuracy_score(y_train, y_train_predict)
print("Training error",Dummy_accuracy)
print("TestError:", Dummy_train_accuracy)


Training error 0.6765971532605097
TestError: 0.6682393841569406


In [41]:
estimators=[("logreg", logreg), ("rf", model_R), ("Dc", Dec_model),("Naive",Model_N)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train, y_train)
#test our model on the test data
print("Voting Classifier",ensemble.score(X_test, y_test))

Voting Classifier 0.8682555445216815


In [44]:
G_Model=GradientBoostingClassifier(random_state=42) #look for the base alg !!
G_Model.fit(X_train,y_train)
G_predict=G_Model.predict(X_test)
G_predict_train=G_Model.predict(X_train)
G_accracy=accuracy_score(y_test,G_predict)
G_accracy_train=accuracy_score(y_train,G_predict_train)
print("Gradient ", G_accracy)
print("Train Gradient ", G_accracy_train)

Gradient  0.8702416418404502
Train Gradient  0.8659879149077063


In [5]:
# OneR algorithm
# not in scikit-learn, so we need to flesh it out ourselves

# Let's perform OneR Algorithm

# best rule and accuracy
best_rule = None
accuracy = 0.0

# loop through each feature:
# calculate the frequency table and compute the accuracy og the rule
for feature in cat_feature:
    frequency_table = train_data.groupby([feature, 'SUSPECT_ARRESTED_FLAG']).size().unstack()
    rule_accuracy = frequency_table.max(axis=1).sum() / frequency_table.sum().sum()

    if rule_accuracy > best_accuracy:
        best_accuracy = rule_accuracy
        best_rule = (feature, frequency_table.idxmax().to_dict())

# Step 4: Print the best rule and its accuracy
print("Best Rule:")
print(f"Feature: {best_rule[0]}")
print(f"Value: {best_rule[1]}")
print(f"Accuracy: {best_accuracy}")

# Step 5: Apply the best rule to make predictions on the testing set
prediction = X_test[best_rule[0]] == best_rule[1][best_rule[0]]

# Step 6: Calculate and print the overall accuracy on the testing set
overall_accuracy = accuracy_score(y_test, prediction)
print(f"Overall Accuracy: {overall_accuracy}")



In [43]:
models = [RandomForestClassifier(random_state=42),RandomForestClassifier(random_state=42),
            LogisticRegression(random_state=0,max_iter=1000),DecisionTreeClassifier(random_state=42),MultinomialNB()
         ]
for model in models:
    print(model, '\n')
    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()
    print('Cross-Validation Accuracy:', score, '\n', '\n')


RandomForestClassifier(random_state=42) 

Cross-Validation Accuracy: 0.8620978746023613 
 

RandomForestClassifier(random_state=42) 

Cross-Validation Accuracy: 0.8620978746023613 
 

LogisticRegression(max_iter=1000, random_state=0) 

Cross-Validation Accuracy: 0.8614354859480446 
 

DecisionTreeClassifier(random_state=42) 

Cross-Validation Accuracy: 0.860938762956651 
 

MultinomialNB() 

Cross-Validation Accuracy: 0.8336231426402934 
 

